# Capstone Project - Mateus Mendelson

This notebook will be used to implement the project of the Applied Data Science Capstone course at Coursera, by IBM.

In [1]:
import pandas as pd
import numpy as np

print('Hello Capstone Project Course')

Hello Capstone Project Course
